In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer,biasLayer,SincBatchLeaky
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss,LayerNorm,BatchNorm1d
from torch.utils.data import DataLoader
from torch.optim import Adam

In [55]:
## change ##
batch_size = 16
clip = .1
block = ResidualBlock
normalFun = normalize2
name = 'raw_sincNet'

In [3]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalFun(train[:,0])
# without tanh is worse-off

Prepare data

In [7]:
# train_gen = SequenceGen(train)
# train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

# val_gen = SequenceGen(train,False)
# val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

In [4]:
train_gen = SequenceGenSample(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSample(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [67]:
model = Sequential(   LayerNorm(37500),
                      SincBatchLeaky(1,64,128,sample_rate=1000000,stride=8),
                      block(64),
                      block(128),                 
                      LambdaLayer(lambda x:x.transpose(1,2)),
                      Dropout(0.5),
                      Linear(256,1,bias=False),
                      LeakyReLU(0.1,True),                  
                      LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),                 
                      biasLayer((1,)),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [ ]:
# model = Sequential(   LayerNorm(37500),
#                       SincConv_fast(64,128,sample_rate=100000,stride=64),
#                       LayerNorm(584),
#                       nn.LeakyReLU(0.1,inplace=True),
#                       nn.Conv1d(64,64,5),
#                       LayerNorm(580),
#                       nn.LeakyReLU(0.1,inplace=True),
#                       nn.Conv1d(64,128,5,stride=2),
#                       LayerNorm(288),
#                       nn.LeakyReLU(0.1,inplace=True),
#                       nn.Conv1d(128,256,5,stride=2),   
#                       LayerNorm(142),
#                       nn.LeakyReLU(0.1,inplace=True),                   
#                       LambdaLayer(lambda x:x.transpose(1,2)),
#                       Dropout(0.5),
#                       Linear(256,1,bias=False),
#                       LeakyReLU(0.2,True),
#                       LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),                 
#                       biasLayer((1,)),
#                       LambdaLayer(lambda x:x.squeeze(1))
#                       ).to('cuda:0')

In [59]:
# model = Sequential(SincConv_fast2(64,128,sample_rate=100000,stride=64),
#                       block(64),
#                       Dropout(0.5),
#                       block(128),
#                       Dropout(0.5),                 
#                       LambdaLayer(lambda x:x.transpose(1,2)),
#                       Linear(256,1,bias=False),
#                       LeakyReLU(0.2,True),
#                       LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),
#                       #LambdaLayer(lambda x:x.squeeze(2).median(1,keepdim=True)[0]),                   
#                       biasLayer((1,)),
#                       #Linear(1,1),
#                       LambdaLayer(lambda x:x.squeeze(1))
#                       ).to('cuda:0')

In [68]:
opt = Adam(trainable_parameter(model),lr=4e-4)

In [69]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [59]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:3.5968757185798426, val_loss:2.648303747177124
epoch:1, train_loss:2.2223448999799214, val_loss:2.8477299213409424
epoch:2, train_loss:2.1272364654220066, val_loss:2.5932843685150146
epoch:3, train_loss:2.0244244910203495, val_loss:2.472490072250366
epoch:4, train_loss:1.9287010563107638, val_loss:2.3878142833709717
Training completed in 32.20494079589844s


Save Model/Validation

In [61]:
save_ModelandValidation_sample(model,val_gen,name)

Submission

In [62]:
make_submission_sample(name,model,normalFun,batch_size)